In [1]:
import numpy as np
import operator
# from numpy.linalg import norm
from numpy.linalg import inv
from numpy.linalg import det
import math
from functools import reduce
import pandas as pd
# %matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('seaborn-whitegrid')
import functools
from ipywidgets import FloatProgress
from IPython.display import display

norm = lambda A: np.linalg.norm(A, ord=2)

### Решение СЛАУ методом простой итерации и методом релаксации

In [128]:
def get_H(n, precision):
    # функция строит матрицу Гильберта размером n x n и округляет значения до precision знаков после запятой
    a = lambda i, j: round(1 / (i + j + 1), precision)
    return np.array([[a(i, j) for j in range(n)] for i in range(n)])

#### Метод простой итерации

In [269]:
def check_ddm(A):
    # функция для проверки, что матрица обладает диагональным преобладанием
    n = len(A)
    for i in range(n):
        s = sum(np.abs(A[i][0:i])) + sum(np.abs(A[i][i + 1:n]))
        if abs(A[i][i]) < s:
            return False

    return True


def find_eigenvalues_bounds(A):
    # находим оценки для собственных чисел
    n = len(A)
    m = A[0][0]
    M = A[0][0]
    for i in range(n):
        r = sum(np.abs(A[i][0:i])) + sum(np.abs(A[i][i + 1:n]))
        m = min(m, A[i][i] - r)
        M = max(M, A[i][i] + r)
    return m, M


def eval_precision(B, x_curr, x_prev):
    # апостериорная оценка погрешности
    return norm(B) / (1 - norm(B)) * norm(x_curr - x_prev)


def iterative_method(A, b, eps):
    # метод простой итерации
    n = len(b)
    if check_ddm(A):
        B = np.array([[0 if i == j else -A[i][j] / A[i][i] for j in range(n)] for i in range(n)])
        c = np.array([b[i] / A[i][i] for i in range(n)])
    elif (A == (A.conjugate()).T).all() and (np.linalg.eig(A)[0] > 0).all():
        m, M = find_eigenvalues_bounds(A)
        m = max(m, 0)
        alpha = 2 / (m + M)
        B = np.eye(n) - alpha * A
        c = alpha * b
    else:
        print("Wrong matrix!")
        return
    
    x_prev = np.ones(n)
    x_curr = B @ x_prev + c

    iters = 0

    while eval_precision(B, x_curr, x_prev) > eps:
        x_prev, x_curr = x_curr, B @ x_curr + c
        iters += 1
    
    print("iters", iters)

    return x_curr


#### Метод релаксации

In [242]:
def relaxation_method(A, b, eps):
    n = len(A)
    x = np.zeros(n)
    iters = 0
    while True:
        for index in np.random.choice(range(n), n):
            if A[index][index] == 0:
                continue
            x[index] = (b[index] - sum([A[index][k] * x[k] for k in range(index)]) - sum([A[index][k] * x[k] for k in range(index + 1, n)])) / A[index][index]

        r = b - A @ x

        if max(abs(r)) < eps:
            break

        if iters % 2 == 0:
            print(f"{max(abs(r)):.5e}", flush=True, end='\r')
        iters += 1

    print(end='\r')
    print()
    print("iters:", iters)
    return x

In [221]:
def relaxation_method2(A, b, eps):
    n = len(A)
    x = np.zeros(n)
    iters = 0
    while True:        
        best_x = x
        for x_i, r_i in zip(range(n), range(n)):
            tmp_x = x
            if A[r_i][x_i] == 0:
                continue

            tmp_x[x_i] = (b[r_i] - sum([A[r_i][k] * x[k] for k in range(x_i)]) - sum([A[r_i][k] * x[k] for k in range(x_i + 1, n)])) / A[r_i][x_i]
            if norm(b - A @ tmp_x) < norm(b - A @ best_x):
                best_x = tmp_x
        
        x = best_x
        r = b - A @ x

        if max(abs(r)) < eps:
            break
        
        if iters % 2 == 0:
            print(f"{max(abs(r)):.5e}", flush=True, end='\r')
        iters += 1
    
    print(end='\r')
    print()
    print("iters:", iters)
    return x

***
### Тесты

In [73]:
%%time
n = 4
matrix = get_H(n, 5)
x_0 = np.random.random_sample(n) * 10 - 5
# x_0 = np.ones(n) 

b = matrix @ x_0
for eps in np.logspace(-2, -5, 6):
    print(f"eps: {eps:.0e}  ", end="")
    x = iterative_method(matrix, b, eps)

eps: 1e-02  iters 67251
eps: 3e-03  iters 82523
eps: 6e-04  iters 97795
eps: 2e-04  iters 113066
eps: 4e-05  iters 128338
eps: 1e-05  iters 143610
CPU times: user 48.9 s, sys: 269 ms, total: 49.2 s
Wall time: 49.1 s


In [272]:
%%time
n = 4
matrix = get_H(n, 5)
x_0 = np.random.random_sample(n) * 10 - 5
# x_0 = np.ones(n) * 2
b = matrix @ x_0
eps = 1e-7

x = iterative_method(matrix, b, eps)

print(x_0)
print(x)
print(norm(x - x_0))

iters 183588
[-1.01955745  2.37995406 -3.1750827  -3.24548244]
[-1.01955744  2.37995402 -3.17508262 -3.24548249]
1.0000062904536809e-07
CPU times: user 14.6 s, sys: 94.3 ms, total: 14.7 s
Wall time: 14.7 s


In [271]:
%%time
n = 4
matrix = np.array([[4, 1, 0, 0], [3, 7, -1, 0], [0, 2, 10, 1], [0, 0, 1, -2]])
x_0 = np.random.random_sample(n) * 10 - 5
# x_0 = np.ones(n) * 2
b = matrix @ x_0
eps = 1e-7

x = iterative_method(matrix, b, eps)

print(x_0)
print(x)
print(norm(x - x_0))

iters 15
[-1.56821984  2.29049707 -0.61427755 -4.40322103]
[-1.56821983  2.29049707 -0.61427755 -4.40322103]
1.3928907960544109e-08
CPU times: user 2.75 ms, sys: 953 µs, total: 3.7 ms
Wall time: 3.93 ms


In [288]:
%%time
n = 4
matrix = get_H(n, 20)
np.random.seed(seed=123)
x_0 = np.random.random_sample(n) * 10 - 5
# x_0 = np.ones(n) 

b = matrix @ x_0
eps = 1e-5
x = iterative_method(matrix, b, eps)

print("error:", norm(x - x_0))

iters 129356
error: 9.999951840140794e-06
CPU times: user 10.3 s, sys: 0 ns, total: 10.3 s
Wall time: 10.4 s


In [289]:
%%time
matrix = get_H(4, 20)
n = len(matrix)


np.random.seed(seed=123)
x_0 = np.random.random_sample(n) * 10 - 5
b = matrix @ x_0
eps = 1e-10
x = relaxation_method2(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))

1.00040e-10
iters: 14911
error: 1.0630055634128142e-06
biggest res: 9.994272076596644e-11
CPU times: user 5.71 s, sys: 763 ms, total: 6.47 s
Wall time: 9.71 s


***
### Большие разреженные матрицы

In [260]:
def get_symmetric(n, rng=1, density=0, seed=None):
    np.random.seed(seed=seed)
    A = np.zeros((n, n))

    for i in range(n):
        for e in range(i + 1, n):
            if np.random.choice([0, 1], p=[1 - density, density]):
                c = np.random.rand() * rng * 2 - rng
                A[i][e] = c
                A[e][i] = c

    for i in range(n):
        A[i][i] = sum(abs(A[i][:])) + np.random.rand() * rng

    return A

In [233]:
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)
get_symmetric(8, density=0.3, seed=123, rng=20)

array([[ 11.46893114,   0.        ,   0.        ,   0.        ,   0.        ,  -3.0757416 ,   7.39318954,   0.        ],
       [  0.        ,  16.15744099,   0.        ,   0.        ,  -2.45711021,   0.        ,   0.        , -12.70033078],
       [  0.        ,   0.        ,   9.97821299,   0.        ,   0.        ,   0.        ,   0.        ,   8.97821299],
       [  0.        ,   0.        ,   0.        ,   8.08164345,   0.        ,  -7.08164345,   0.        ,   0.        ],
       [  0.        ,  -2.45711021,   0.        ,   0.        ,   3.45711021,   0.        ,   0.        ,   0.        ],
       [ -3.0757416 ,   0.        ,   0.        ,  -7.08164345,   0.        ,  11.15738504,   0.        ,   0.        ],
       [  7.39318954,   0.        ,   0.        ,   0.        ,   0.        ,   0.        ,   8.39318954,   0.        ],
       [  0.        , -12.70033078,   8.97821299,   0.        ,   0.        ,   0.        ,   0.        ,  22.67854378]])

In [261]:
%%time

matrix = get_symmetric(300, rng=10000, density=0.2, seed=256)
n = len(matrix)

assert det(matrix) != 0, "det is zero"
assert (matrix == (matrix.conjugate()).T).all() and (np.linalg.eig(matrix)[0] > 0).all(), "bad matrix"

np.random.seed(seed=127)
x_0 = np.random.rand(n) * 100 - 50
b = matrix @ x_0
eps = 1e-8
x = relaxation_method2(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))

/mnt/sdb1/university/6/calcs practices/hw/venv/lib/python3.8/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


1.30385e-08
iters: 15
error: 8.838676222867268e-14
biggest res: 7.450580596923828e-09
CPU times: user 4.46 s, sys: 2.21 s, total: 6.67 s
Wall time: 2.96 s


In [262]:
%%time

matrix = get_symmetric(300, rng=10000, density=0.2, seed=256)
n = len(matrix)

assert det(matrix) != 0, "det is zero"
assert (matrix == (matrix.conjugate()).T).all() and (np.linalg.eig(matrix)[0] > 0).all(), "bad matrix"

np.random.seed(seed=127)
x_0 = np.random.rand(n) * 100 - 50
b = matrix @ x_0
eps = 1e-8
x = relaxation_method(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))

1.76951e-08
iters: 56
error: 8.497531641349451e-14
biggest res: 7.450580596923828e-09
CPU times: user 6.73 s, sys: 2.98 s, total: 9.71 s
Wall time: 4.15 s


In [263]:
%%time

matrix = get_symmetric(300, rng=3000, density=0.2, seed=145)
n = len(matrix)

assert det(matrix) != 0, "det is zero"
assert (matrix == (matrix.conjugate()).T).all() and (np.linalg.eig(matrix)[0] > 0).all(), "bad matrix"

np.random.seed(seed=14)
x_0 = np.random.rand(n) * 100 - 50
b = matrix @ x_0
eps = 1e-8
x = relaxation_method(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))

4.37722e-08
iters: 55
error: 1.535299163405393e-13
biggest res: 4.6566128730773926e-09
CPU times: user 7.15 s, sys: 2.67 s, total: 9.82 s
Wall time: 3.97 s


In [264]:
%%time

matrix = get_symmetric(400, rng=10000, density=0.3, seed=1024)
n = len(matrix)

assert det(matrix) != 0, "det is zero"
assert (matrix == (matrix.conjugate()).T).all() and (np.linalg.eig(matrix)[0] > 0).all(), "bad matrix"

np.random.seed(seed=37)
x_0 = np.random.rand(n) * 100 - 50
b = matrix @ x_0
eps = 1e-6
x = relaxation_method2(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))

1.61305e-06
iters: 11
error: 4.747383273094868e-13
biggest res: 8.195638656616211e-08
CPU times: user 6.55 s, sys: 2.82 s, total: 9.37 s
Wall time: 4.31 s


In [265]:
%%time

matrix = get_symmetric(400, rng=10000, density=0.3, seed=1024)
n = len(matrix)

assert det(matrix) != 0, "det is zero"
assert (matrix == (matrix.conjugate()).T).all() and (np.linalg.eig(matrix)[0] > 0).all(), "bad matrix"

np.random.seed(seed=37)
x_0 = np.random.rand(n) * 100 - 50
b = matrix @ x_0
eps = 1e-6
x = relaxation_method(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))

4.46849e-06
iters: 46
error: 2.6723868582567984e-12
biggest res: 7.413327693939209e-07
CPU times: user 9.43 s, sys: 3.71 s, total: 13.1 s
Wall time: 5.33 s


In [301]:
%%time

matrix = get_symmetric(400, rng=10000, density=0.3, seed=1024)
n = len(matrix)

assert det(matrix) != 0, "det is zero"
assert (matrix == (matrix.conjugate()).T).all() and (np.linalg.eig(matrix)[0] > 0).all(), "bad matrix"

np.random.seed(seed=37)
x_0 = np.random.rand(n) * 100 - 50
b = matrix @ x_0
eps = 1e-12

x = iterative_method(matrix, b, eps)

print("error:", norm(x - x_0))
print("biggest res:", max(abs(b - matrix @ x)))
print("error:", norm(x - x_0))

iters 19
error: 8.381292669139096e-13
biggest res: 6.705522537231445e-08
error: 8.381292669139096e-13
CPU times: user 4.71 s, sys: 1.25 s, total: 5.95 s
Wall time: 2.97 s
